In [1]:
# import os
# import pandas as pd

# temp_list_df = []
# dir_path = os.path.join(os.getcwd(),'data_scrap')

# for path in os.listdir(dir_path):
#     if os.path.isfile(os.path.join(dir_path, path)):
#         temp_list_df.append(pd.read_csv(os.path.join(dir_path,path)))

# df = pd.concat(temp_list_df)
# df.to_csv('data_scrap_combined.csv',index=False)

In [2]:
import pandas as pd
import os
import re
import string

In [3]:
df = pd.read_csv('data_scrap_combined.csv')

In [4]:
def text_cleaning(text:str)->str:
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    # text = re.sub('\w*\d\w*', '', text)
    return text

def price_to_int(text:str):
    text = text.replace('Rp','').replace('.','')
    return int(text)

In [5]:
df['text_cleaned'] = df['text'].apply(lambda x:text_cleaning(x))
df['price_int'] = df['price'].apply(lambda x:price_to_int(x))

In [6]:
df_cleaned = df[['text_cleaned','price_int','type_']]

In [7]:
df_cleaned['type_']

0         aksesoris-handphone
1         aksesoris-handphone
2         aksesoris-handphone
3         aksesoris-handphone
4         aksesoris-handphone
                 ...         
126445       wearable-devices
126446       wearable-devices
126447       wearable-devices
126448       wearable-devices
126449       wearable-devices
Name: type_, Length: 126450, dtype: object

In [8]:
import os
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
#import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)

In [9]:
unique_col = df_cleaned['type_'].unique()
def to_category_id(x, unique_col):
    count = 0
    for col in unique_col:
        if str(x) ==  str(col):
            return count
        count+=1
df_cleaned['category_id'] = df_cleaned['type_'].apply(lambda x : to_category_id(x,unique_col))

C:\Users\Andrey\AppData\Local\Temp\ipykernel_12140\1051128527.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['category_id'] = df_cleaned['type_'].apply(lambda x : to_category_id(x,unique_col))


In [10]:
tfidf = TfidfVectorizer(sublinear_tf=True, 
                        min_df=5,
                        max_features=10000)

# We transform each complaint into a vector
features = tfidf.fit_transform(df_cleaned['text_cleaned']).toarray()

labels = df_cleaned['category_id']

In [11]:
# 17 minutes
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

# 5 Cross-validation
CV = 2
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

c:\Users\Andrey\miniconda3\envs\rakamin\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Andrey\miniconda3\envs\rakamin\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [13]:
X = df_cleaned['text_cleaned'] # Collection of documents
y = df_cleaned['type_'] # Target or the labels we want to predict (i.e., the 13 different complaints of products)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

fitted_vectorizer = tfidf.fit(X_train)
tfidf_vectorizer_vectors = fitted_vectorizer.transform(X_train)

model = LogisticRegression().fit(tfidf_vectorizer_vectors, y_train)

c:\Users\Andrey\miniconda3\envs\rakamin\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
from sklearn.metrics import classification_report

y_pred = model.predict(fitted_vectorizer.transform(X_test))
print(classification_report(y_test, y_pred, target_names=model.classes_))

                           precision    recall  f1-score   support

   aksesoris-game-console       0.84      0.81      0.83      1417
      aksesoris-handphone       0.80      0.95      0.87      2327
aksesoris-komputer-laptop       0.86      0.82      0.84      1484
  aksesoris-mobile-gaming       0.86      0.86      0.86      1567
      aksesoris-pc-gaming       0.90      0.86      0.88      1516
         aksesoris-tablet       0.86      0.62      0.72      1261
   alat-pendingin-ruangan       0.97      0.97      0.97      1397
                    audio       0.84      0.94      0.89      1442
                  cd-game       0.88      0.92      0.90      1526
          desktop-mini-pc       0.93      0.88      0.90      1446
         elektronik-dapur       0.96      0.96      0.96      1339
  elektronik-rumah-tangga       0.96      0.95      0.96      1496
             game-console       0.81      0.82      0.81      1387
                handphone       0.88      0.89      0.88     